In [1]:
# SecureString parameter stored in AWS System Manager that holds a GitHub personal access token.
parameter_name = 'github_token'

# Repositories for which reports will be generated.
#repo_names = ['aws-amplify/amplify-android', 'aws-amplify/aws-sdk-android']
repo_names = ['aws-amplify/amplify-ios', 'aws-amplify/aws-sdk-ios', 'awslabs/aws-mobile-appsync-sdk-ios']


In [2]:
# Retrieve the GitHub token from SSM to prevent oops-I-pushed-credentials-to-GitHub uh-ohs.

import boto3
from github import Github

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
token = response['Parameter']['Value']
github = Github(token)

In [3]:
# Loop through each repository, grab all issues, and create a DataFrame for each.

import pandas as pd

repos = {}

for repo_name in repo_names:
    repo = github.get_repo(repo_name)
    issues = []

    for issue in repo.get_issues(state='all'):
        labels = pd.array([label.name for label in issue.labels])
        
        if issue.pull_request is None:
            issues.append([labels, issue.created_at, issue.closed_at])
        
    repos[repo_name] = pd.DataFrame(issues, columns=['labels', 'created_at', 'closed_at'])

In [4]:
# Using the DataFrames, process and grab the counts of issues. Go through each label and gets counts for each label and other applied labels.

from datetime import datetime
from IPython.display import display, HTML

seven_days_ago = datetime.now() - pd.Timedelta('7 days')
thirty_days_ago = datetime.now() - pd.Timedelta('30 days')

open_counts = []
label_counts = []

for repo_name in repo_names:
    r = repos[repo_name]
    labels = {label for labels in repos[repo_name].labels for label in labels}

    open_issues = r[~(r.closed_at > '1970-01-01')]
    last_week_open_issues = r[(r.created_at < seven_days_ago) & ~(r.closed_at < seven_days_ago)]
    last_month_open_issues = r[(r.created_at < thirty_days_ago) & ~(r.closed_at < thirty_days_ago)]
    open_issues_count = len(open_issues.index)
    last_week_open_issues_count = len(last_week_open_issues.index)
    last_month_open_issues_count = len(last_month_open_issues.index)
    
    open_counts.append([repo_name,
                        open_issues_count,
                        last_week_open_issues_count,
                        open_issues_count - last_week_open_issues_count,
                        last_month_open_issues_count,
                        open_issues_count - last_month_open_issues_count])
    
    for label in sorted(labels):
        label_mask = open_issues.labels.apply(lambda l: label in l)
        issues = open_issues[label_mask]
        last_week_label_issues = issues[(issues.created_at < seven_days_ago) & ~(issues.closed_at < seven_days_ago)]
        last_month_label_issues = issues[(issues.created_at < thirty_days_ago) & ~(issues.closed_at < thirty_days_ago)]
        label_issues_count = len(issues.index)
        last_week_label_issues_count = len(last_week_label_issues.index)
        last_month_label_issues_count = len(last_month_label_issues.index)
        
        if label_issues_count or last_week_label_issues_count or last_month_label_issues_count:
            label_counts.append([repo_name,
                                 label,
                                 '',
                                 label_issues_count,
                                 last_week_label_issues_count,
                                 label_issues_count - last_week_label_issues_count,
                                 last_month_label_issues_count,
                                 label_issues_count - last_month_label_issues_count])

            for other_label in labels - {label}:
                other_label_mask = issues.labels.apply(lambda l: other_label in l)
                other_label_issues = issues[other_label_mask]

                if not other_label_issues.empty:
                    last_week_other_label_issues = other_label_issues[(other_label_issues.created_at < seven_days_ago) & ~(other_label_issues.closed_at < seven_days_ago)]
                    last_month_other_label_issues = other_label_issues[(other_label_issues.created_at < thirty_days_ago) & ~(other_label_issues.closed_at < thirty_days_ago)]
                    other_label_issues_count = len(other_label_issues.index)
                    last_week_other_label_issues_count = len(last_week_other_label_issues.index)
                    last_month_other_label_issues_count = len(last_month_other_label_issues.index)

                    if other_label_issues_count or last_week_other_label_issues_count or last_month_other_label_issues_count:
                        label_counts.append([repo_name,
                                             label,
                                             other_label,
                                             other_label_issues_count,
                                             last_week_other_label_issues_count,
                                             other_label_issues_count - last_week_other_label_issues_count,
                                             last_month_other_label_issues_count,
                                             other_label_issues_count - last_month_other_label_issues_count])
                
open_df = pd.DataFrame(open_counts, columns=['Repo', 'Open', 'Last Week', 'WoW', 'Last Month', 'MoM'])
labels_df = pd.DataFrame(label_counts, columns=['Repo', 'Label', 'Other Label', 'Open', 'Last Week', 'WoW', 'Last Month', 'MoM'])

In [5]:
# Build the report for display.

pd.set_option('display.max_rows', 500)

display(HTML('<h1>Open Source – Ops Report</h1>'))
display(HTML(f'<em>Generated on {datetime.now()}'))

display(HTML('<h2>Open Issues</h2>'))
display(open_df)

display(HTML('<h2>Open Issues by Label</h2>'))

for repo_name in repo_names:
    display(HTML(f'<h3>{repo_name}</h3>'))
    display(labels_df[labels_df.Repo == repo_name])

,Repo,Open,Last Week,WoW,Last Month,MoM
0,aws-amplify/amplify-ios,53,55,-2,70,-17
1,aws-amplify/aws-sdk-ios,183,184,-1,169,14
2,awslabs/aws-mobile-appsync-sdk-ios,65,65,0,62,3


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
0,aws-amplify/amplify-ios,API,,8,8,0,7,1
1,aws-amplify/amplify-ios,API,P0,2,2,0,2,0
2,aws-amplify/amplify-ios,API,enhancement,2,2,0,2,0
3,aws-amplify/amplify-ios,API,bug,2,2,0,2,0
4,aws-amplify/amplify-ios,API,DataStore,4,4,0,3,1
5,aws-amplify/amplify-ios,Analytics,,1,1,0,1,0
6,aws-amplify/amplify-ios,Analytics,Requesting Feedback,1,1,0,1,0
7,aws-amplify/amplify-ios,Build,,3,3,0,2,1
8,aws-amplify/amplify-ios,Build,DataStore,1,1,0,0,1
9,aws-amplify/amplify-ios,Build,Core,1,1,0,1,0


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
70,aws-amplify/aws-sdk-ios,APIGateway,,8,8,0,6,2
71,aws-amplify/aws-sdk-ios,APIGateway,Bug,1,1,0,1,0
72,aws-amplify/aws-sdk-ios,APIGateway,Cognito,2,2,0,0,2
73,aws-amplify/aws-sdk-ios,APIGateway,Feature Request,4,4,0,4,0
74,aws-amplify/aws-sdk-ios,APIGateway,Investigating,1,1,0,1,0
75,aws-amplify/aws-sdk-ios,APIGateway,Question,1,1,0,1,0
76,aws-amplify/aws-sdk-ios,AWSMobileClient,,39,39,0,32,7
77,aws-amplify/aws-sdk-ios,AWSMobileClient,Bug,5,5,0,5,0
78,aws-amplify/aws-sdk-ios,AWSMobileClient,Cognito,11,11,0,7,4
79,aws-amplify/aws-sdk-ios,AWSMobileClient,Feature Request,22,22,0,22,0


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
280,awslabs/aws-mobile-appsync-sdk-ios,AppSync,,57,57,0,56,1
281,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Build,1,1,0,1,0
282,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Bug,20,20,0,20,0
283,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Good First Issue,2,2,0,2,0
284,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Pending,4,4,0,4,0
285,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Help Wanted,1,1,0,1,0
286,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Tests,1,1,0,1,0
287,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Feature Request,19,19,0,19,0
288,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Service,2,2,0,2,0
289,awslabs/aws-mobile-appsync-sdk-ios,AppSync,Product Review,1,1,0,1,0
